In [57]:
from ase.io import read # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from ase.geometry.analysis import Analysis # type: ignore
import ase
from ase import Atoms
from graph_tool.all import Graph, GraphView, label_components
import itertools
BOND_DISTANCE = 2.0

In [58]:
data = read("my-first-input:/dump.lammpstrj", index=":")

In [80]:
frame = data[144]

In [81]:
def get_bonded_atoms(frame):
    distances = frame.get_all_distances(mic=True)
    return [
        (i, j)
        for i, j in list(zip(*np.where(distances <= BOND_DISTANCE)))
        if i != j and i < j
    ]

In [82]:
get_bonded_atoms(frame)

[(0, 32),
 (2, 98),
 (4, 66),
 (4, 82),
 (7, 55),
 (12, 54),
 (12, 69),
 (17, 49),
 (19, 90),
 (20, 96),
 (22, 39),
 (22, 45),
 (23, 25),
 (23, 93),
 (24, 56),
 (25, 88),
 (25, 93),
 (27, 96),
 (30, 67),
 (31, 41),
 (37, 83),
 (39, 45),
 (40, 66),
 (46, 84),
 (47, 84),
 (54, 69),
 (70, 73)]

In [83]:
def get_bond(types, bonds, i=None, j=None):
    my_bonds = []
    for b in bonds:
        if i in b:
            my_bonds.append(b)

    if len(my_bonds) == 0:
        return types[i]
        
    o_atoms = []
    for b in my_bonds:
        for k in b:
            if k != i:
                o_atoms.append(k)
        
    if len(my_bonds) == 1:
        return "{}{}".format(types[i], types[o_atoms[0]])
        
    if len(my_bonds) == 2:
        if (o_atoms[0], o_atoms[1]) in bonds or (o_atoms[1], o_atoms[0]) in bonds:
            print("Existing bond detected: {}-{}".format(types[o_atoms[0]], types[o_atoms[1]]))
            return 'c{}{}{}'.format(types[o_atoms[0]][0], types[i][0], types[o_atoms[1]][0])
        print("No existing bond between {} and {}".format(types[o_atoms[0]], types[o_atoms[1]]))
        return "{}{}{}".format(
            get_bond(types, bonds, o_atoms[0], i)[0],
            types[i][0],
            get_bond(types, bonds, o_atoms[1], i)[0],
        )
        
    if len(my_bonds) == 3:
        return "{}{}({}){}".format(
            get_bond(types, bonds, o_atoms[0], i)[0],
            types[i][0],
            get_bond(types, bonds, o_atoms[1], i)[0],
            get_bond(types, bonds, o_atoms[2], i)[0],
        )
        
    else:
        return None


In [128]:
bonds = get_bonded_atoms(frame)
# bond_arr = []
# types = ["A" if n == 1 else "B" for n in frame.numbers]
# for i in range(len(frame)):
#     bond_arr.append(get_bond(types, bonds, i))

In [129]:
bonds

[(4, 66),
 (7, 15),
 (7, 41),
 (7, 55),
 (7, 80),
 (12, 69),
 (15, 80),
 (17, 78),
 (24, 56),
 (26, 95),
 (28, 85),
 (28, 89),
 (29, 50),
 (33, 70),
 (37, 52),
 (37, 83),
 (37, 97),
 (41, 55),
 (42, 84),
 (47, 63),
 (52, 83),
 (52, 97),
 (63, 84)]

In [87]:
neighbors_graph = Graph(directed=False)
neighbors_graph.add_edge_list(bonds)

In [88]:
components = label_components(neighbors_graph)[0]

In [89]:
components

<VertexPropertyMap object with value type 'int32_t', for Graph 0x17ce7bad0, at 0x17cee7d90>

In [90]:
graph_bonds = [GraphView(neighbors_graph, 
                         vfilt=components.a == i).get_vertices()
            for i in components.a]

In [130]:
for atoms in graph_bonds:
    if len(atoms) == 1:
        molecule = types[atoms[0]]
        #print(molecule)
    elif len(atoms) == 2:
        molecule = "{}{}".format(types[atoms[0]], types[atoms[1]])
        #print(molecule)
    else:
        distances = {}
        for pair in itertools.combinations(atoms, 2):
            s1 = frame.positions[pair[0]]
            s2 = frame.positions[pair[1]]
            distance = np.linalg.norm(s1 - s2)
            distances[pair] = distance
        #print(distances)
        
        sorted_atoms = sorted(atoms, key=lambda x: distances.get(x, 0))
        outer_atoms = max(distances, key=distances.get)  
        middle_atoms = [atom for atom in sorted_atoms if atom not in outer_atoms]
        middle_atoms.sort(key=lambda x: -distances.get((x, outer_atoms[0]), float('inf')))
        for atom in middle_atoms:
            sorted_atoms.remove(atom)
        for atom in middle_atoms:
            sorted_atoms.insert(len(sorted_atoms) - 1, atom)
        
        molecule = ''.join(types[a] for a in sorted_atoms)
        #print(molecule)


In [131]:
from ase.io import read # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from ase.geometry.analysis import Analysis # type: ignore
import ase # type: ignore
from ase import Atoms # type: ignore
from graph_tool.all import Graph, GraphView, label_components # type: ignore
import itertools # type: ignore

BOND_DISTANCE = 2.0

def get_bonded_atoms(frame):
    distances = frame.get_all_distances(mic=True)
    return [
        (i, j)
        for i, j in list(zip(*np.where(distances <= BOND_DISTANCE)))
        if i != j and i < j
    ]

def get_molecules_in_frame(frame):
    bonds = get_bonded_atoms(frame)
    types = ["A" if n == 1 else "B" for n in frame.numbers]
    neighbors_graph = Graph(directed=False)
    neighbors_graph.add_edge_list(bonds)
    components = label_components(neighbors_graph)[0]
    graph_bonds = [GraphView(neighbors_graph, 
                            vfilt=components.a == i).get_vertices()
            for i in components.a]
    molecules = []
    for atoms in graph_bonds:
        if len(atoms) == 1:
            molecule = types[atoms[0]]
            molecules.append(molecule)
        elif len(atoms) == 2:
            molecule = "{}{}".format(types[atoms[0]], types[atoms[1]])
            molecules.append(molecule)
        else:
            distances = {}
            for pair in itertools.combinations(atoms, 2):
                s1 = frame.positions[pair[0]]
                s2 = frame.positions[pair[1]]
                distance = np.linalg.norm(s1 - s2)
                distances[pair] = distance
            
            sorted_atoms = sorted(atoms, key=lambda x: distances.get(x, 0))
            outer_atoms = max(distances, key=distances.get)  
            middle_atoms = [atom for atom in sorted_atoms if atom not in outer_atoms]
            middle_atoms.sort(key=lambda x: -distances.get((x, outer_atoms[0]), float('inf')))
            for atom in middle_atoms:
                sorted_atoms.remove(atom)
            for atom in middle_atoms:
                sorted_atoms.insert(len(sorted_atoms) - 1, atom)
            
            molecule = ''.join(types[a] for a in sorted_atoms)
            molecules.append(molecule)
    return molecules, graph_bonds

def get_molecule_type_for_atom(atom_number, molecules_each_frame, graph_bonds_each_frame):
    atom_number = atom_number - 1 
    molecule_types = []
    for frame_number in sorted(molecules_each_frame.keys()):
        molecules = molecules_each_frame[frame_number]
        graph_bonds = graph_bonds_each_frame[frame_number]
        for molecule, atoms in zip(molecules, graph_bonds):
            if atom_number in atoms:
                molecule_types.append((frame_number, molecule))
                break
    return molecule_types

def plot_molecule_types(atom_number, molecule_types_over_time):
    frames = [frame for frame, _ in molecule_types_over_time]
    molecules = [molecule for _, molecule in molecule_types_over_time]

    plt.figure(figsize=(10, 6))
    plt.plot(frames, molecules, linestyle='-', color='b')
    plt.xlabel('Frame Number')
    plt.ylabel('Cluser Type')
    plt.title(f'Cluster Type for Atom {atom_number} Over Time')
    plt.xticks(ticks=range(min(frames), max(frames)+1, 50))
    plt.grid(True)
    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    #data = read("dump.lammpstrj", index=":")
    number = 0
    molecules_each_frame = {}
    graph_bonds_each_frame = {}
    for frame in data:
        molecules_in_frame, graph_bonds_in_frame = get_molecules_in_frame(frame)
        molecules_each_frame[number] = molecules_in_frame
        graph_bonds_each_frame[number] = graph_bonds_in_frame
        number += 1

In [139]:
def get_configuration_changes(molecules_each_frame, graph_bonds_each_frame):
    total_atoms = len(molecules_each_frame[0])
    total_counts = {}
    for atom_num in range(total_atoms):
        counts = {}
        atom_configs = get_molecule_type_for_atom(atom_num + 1, molecules_each_frame, graph_bonds_each_frame)
        for i in range(1, len(atom_configs)):
            config1 = atom_configs[i - 1][1]
            config2 = atom_configs[i][1]
            if config1 != config2:
                if config1 not in counts:
                    counts[config1] = {}
                if config2 not in counts[config1]:
                    counts[config1][config2] = 1
                else:
                    counts[config1][config2] += 1
        for config1, transitions in counts.items():
            if config1 not in total_counts:
                total_counts[config1] = {}
            for config2, count in transitions.items():
                if config2 not in total_counts[config1]:
                    total_counts[config1][config2] = count
                else:
                    total_counts[config1][config2] += count
    return total_counts


In [140]:
total_counts = track_configuration_changes(molecules_each_frame, graph_bonds_each_frame)
total_counts

{'A': {'AA': 127,
  'ABA': 16,
  'AB': 90,
  'AAB': 21,
  'AABB': 5,
  'AAA': 10,
  'ABB': 1,
  'BAAB': 3,
  'ABBA': 3,
  'ABAB': 4,
  'AABBA': 3,
  'AAABAB': 3,
  'AAAB': 1,
  'ABAA': 2,
  'AABA': 1},
 'AA': {'AAB': 6, 'A': 133, 'AAA': 20, 'AABA': 2},
 'AAB': {'AA': 6,
  'ABA': 219,
  'A': 22,
  'AB': 42,
  'AABB': 12,
  'ABAB': 5,
  'BAAB': 11,
  'B': 3},
 'ABA': {'AAB': 222,
  'A': 15,
  'ABAB': 12,
  'AB': 28,
  'AAAB': 2,
  'ABAA': 4,
  'AABA': 2,
  'BAAB': 2,
  'AABB': 3,
  'AAABB': 3,
  'AABAB': 3},
 'AB': {'A': 80,
  'ABA': 31,
  'BAB': 41,
  'ABB': 60,
  'ABAB': 18,
  'ABBAB': 2,
  'BABAB': 2,
  'ABBA': 4,
  'AAB': 41,
  'BAAB': 10,
  'AABB': 8,
  'AABA': 1,
  'ABBB': 2,
  'AABBA': 4,
  'AAABB': 2,
  'AABAB': 2,
  'B': 78},
 'BAB': {'ABB': 165,
  'AB': 52,
  'ABAB': 12,
  'ABBA': 6,
  'BBAB': 10,
  'ABBAB': 3,
  'BABAB': 3,
  'BABB': 3,
  'AABB': 12,
  'B': 23},
 'ABB': {'AB': 53,
  'BAB': 170,
  'A': 3,
  'BAAB': 6,
  'BBAB': 5,
  'ABBA': 3,
  'AABB': 3,
  'BB': 6,
  'B': 26}